In [10]:
from QpixAsic import *
import random
import math

# This is a single trial array

# Seed the RNG
random.seed(None)

# Debug status
debugLevel = 5 #5 - debug

# Create an array of QPixAsics
nRows = 2
nCols = 2
nPixs = 16

# Nominal frequency and associated spread (as 1-sigma in percent)
fNominal  = 50e6
pctSpread = 0.05

# Time interval between DAQ-node timestamps
deltaT = 1.0
# Ticks in nominal oscillator between DAQ-node timestamps
deltaTick = int(math.floor(deltaT * fNominal))
# Small time interval used for stepping simulation almost up to current
timeEpsilon = 1e-6

# How long to run the simulation
timeEnd = 2.5

In [11]:
deltaTick

50000000

In [12]:
# Create the array and populate all positions
asicMatrix = [[None for i in range(nCols)] for j in range(nRows)] 

for i in range(0,nRows):
  for j in range(0,nCols):
    # Create the random frequency here
    fThisAsic = random.gauss(fNominal,fNominal*pctSpread)
    # Create an ASIC at this position
    asicMatrix[i][j] = QPixAsic(fThisAsic, nPixs, row = i, col = j)
    ## Debug info
    if debugLevel >= 5:
      print("Created ASIC at row "+str(i)+" col "+str(j)+" with frequency "+str(fThisAsic))

#if debugLevel >= 5:
#  print(asicMatrix)

Created ASIC at row 0 col 0 with frequency 49089101.435170755
Created ASIC at row 0 col 1 with frequency 48828027.56922385
Created ASIC at row 1 col 0 with frequency 52266090.606685646
Created ASIC at row 1 col 1 with frequency 52056187.980699204


In [13]:
# Create the connections between all ASICs
for i in range(0,nRows):
  for j in range(0,nCols):
    # 0 - North
    # 1 - East
    # 2 - South
    # 3 - West
    if i > 0:
      asicMatrix[i][j].connections[0] = asicMatrix[i-1][j]
    if i < nRows-1:
      asicMatrix[i][j].connections[2] = asicMatrix[i+1][j]
    if j > 0:
      asicMatrix[i][j].connections[3] = asicMatrix[i][j-1]
    if j < nCols-1:
      asicMatrix[i][j].connections[1] = asicMatrix[i][j+1]    

In [14]:
# Create a dummy DAQ node and connect it up to our target
daqNode = QPixAsic(fNominal, 0, isDaqNode = True)
asicMatrix[0][0].connections[3] = daqNode

In [15]:
# Print count of connections vs. position
if debugLevel >= 5:
  print("Connection count per ASIC:")
  for i in range(0,nRows):
    for j in range(0,nCols):
      print( str(asicMatrix[i][j].CountConnections())+" ", end = '')
    print("")

Connection count per ASIC:
3 2 
2 2 


In [16]:
# Print north connections vs. position
if debugLevel >= 5:
  for d in range(0,len(DIRECTIONS)):
    dir = DIRECTIONS[d]
    print("Connections to the "+dir+" per ASIC:")
    for i in range(0,nRows):
      for j in range(0,nCols):
        print( str(asicMatrix[i][j].HasConnection(d))+" ", end = '')
      print("")

Connections to the North per ASIC:
0 0 
1 1 
Connections to the East per ASIC:
1 0 
1 0 
Connections to the South per ASIC:
1 1 
0 0 
Connections to the West per ASIC:
1 1 
0 1 


In [17]:
# Start time now
timeNow = 0
tickNow = 0
stepNum = 0

# Create an empty processing queue
procQueue = ProcQueue()

In [20]:
timeNow += deltaT
tickNow += deltaTick

# Maximum queue depths
maxConnQueueDepths = [[[0 for i in range(nCols)] for j in range(nRows)] for k in range(4)]
maxSelfQueueDepths = [[0 for i in range(nCols)] for j in range(nRows)]

while(timeNow < timeEnd):

  # Make sure there's nothing to do right now
  # Now iterate through ASICs and keep doing processing on them until
  # they're caught up to the latest time.
  for i in range(nRows):
    for j in range(nCols):
      newProcessItems = asicMatrix[i][j].Process(timeNow - timeEpsilon)
      if len(newProcessItems) > 0:
        print("WARNING: ASIC ("+str(i)+","+str(j)+") had things left to do at next major time step")

        # Print states if they were updated
        if debugLevel >= 5:
          asicMatrix[i][j].PrintStatus()
          print("ASIC ("+str(i)+","+str(j)+") state updated in step "+str(stepNum)+", time "+str(asicMatrix[i][j].absTimeNow)+":")
          # for i in range(0,nRows):
          #   for j in range(0,nCols):
          #     print( str(asicMatrix[i][j].stateNum)+" ", end = '')
          #   print("")

  # Add a first timestamp at 1 second
  timestamp = PixelHit(tickNow, [], None, None)                
    
  procQueue.AddQueueItem(asicMatrix[0][0], 3, timestamp, timeNow)

  while(procQueue.Length() > 0):
    #  print("Before step "+str(stepNum)+" list length is: "+str(procQueue.Length()))
    nextItem = procQueue.PopQueue()
    # print("Next item is: ",end='')
    # print(nextItem)
    nextAsic = nextItem.asic
    newQueueItems = nextAsic.ReceiveData(nextItem)
    for item in newQueueItems:
      procQueue.queue.append(item)
      procQueue.SortQueue()
      # print("After step "+str(stepNum)+" list length is: "+str(procQueue.Length()))

    # Check the current queue depths
    for i in range(nRows):
      for j in range(nCols):
        # Local queue depth
        localDepthA = len(asicMatrix[i][j].queues[0])
        localDepthB = len(asicMatrix[i][j].queues[1])
        localMax    = localDepthA if localDepthA > localDepthB else localDepthB
        cur_maxLocalDepth = maxSelfQueueDepths[i][j]
        if (localMax > cur_maxLocalDepth):
          maxSelfQueueDepths[i][j] = localMax
          # Do the same for all the neighbor queues
          for d in range(4):
            cur_maxLocalDepth = maxConnQueueDepths[d][i][j]
            localMax    = len(asicMatrix[i][j].connQueues[d])
            if localMax > cur_maxLocalDepth:
              maxConnQueueDepths[d][i][j] = cur_maxLocalDepth

    # Print states at every step
    # if debugLevel >= 5:
    #   asicMatrix[i][j].PrintStatus()
      # print("ASIC state status after step "+str(stepNum)+", time "+str(nextItem[3])+":")
      # for i in range(nRows):
      #   for j in range(nCols):
          # print(str(asicMatrix[i][j].stateNum)+" ", end = '')

    # Now iterate through ASICs and keep doing processing on them until
    # they're caught up to the latest time.
    somethingToDo = True
    while somethingToDo and procQueue.Length() > 0:
      nextTime = procQueue.queue[0][3]
      somethingToDo = False
      for i in range(nRows):
        for j in range(nCols):
          if nextTime > asicMatrix[i][j].absTimeNow:
            nothingToDo = False
            newProcessItems = asicMatrix[i][j].Process(nextTime)
            for item in newProcessItems:
              procQueue.AddQueueItem(*item)
              # procQueue.SortQueue()

            # Print states if they were updated
            if debugLevel >= 5:
              asicMatrix[i][j].PrintStatus()
              print("ASIC ("+str(i)+","+str(j)+") state updated in step "+str(stepNum)+", time "+str(asicMatrix[i][j].absTimeNow)+":")
              # for i in range(0,nRows):
              #   for j in range(0,nCols):
              #     print(str(asicMatrix[i][j].stateNum)+" ", end = '')

    stepNum += 1

  timeNow += deltaT
  tickNow += deltaTick

In [22]:
print("MAX QUEUE DEPTHS")
# Check the current queue depths
for i in range(0,nRows):
  for j in range(0,nCols):
    print(str(i)+" "+str(j)+" "+str(asicMatrix[i][j]._maxLocalDepth)+" ",end='')
    for d in range(0,4):
      print(str(asicMatrix[i][j].maxConnDepths[d])+" ",end='')
    print()

MAX QUEUE DEPTHS
0 0 6 0 0 0 0 
0 1 0 0 0 0 0 
1 0 0 0 0 0 0 
1 1 0 0 0 0 0 
